In [1]:
!nvidia-smi

Tue Feb 11 05:43:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%time
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4601142 sha256=2c288148e0fb1f4885f5d33db6806ed5d725909328b200913f04c90a08109261
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python
CPU times: user 820 ms, sys: 117 ms, total: 937 ms
Wall time: 2min 36s


In [3]:
from llama_cpp import Llama

In [4]:
!wget https://huggingface.co/ChristianAzinn/labse-gguf/resolve/main/labse.Q8_0.gguf

--2025-02-11 05:46:49--  https://huggingface.co/ChristianAzinn/labse-gguf/resolve/main/labse.Q8_0.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.34, 13.35.202.97, 13.35.202.40, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.34|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/16/de/16de140bf5e3123a1aea5c5d6cde9199378efa2f30553724d633c8cd3228a51f/89ec00269f2d30015194e49f0b6b1a2b9affe6d14c46762c4a0edd425f41a171?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27labse.Q8_0.gguf%3B+filename%3D%22labse.Q8_0.gguf%22%3B&Expires=1739256409&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTI1NjQwOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzE2L2RlLzE2ZGUxNDBiZjVlMzEyM2ExYWVhNWM1ZDZjZGU5MTk5Mzc4ZWZhMmYzMDU1MzcyNGQ2MzNjOGNkMzIyOGE1MWYvODllYzAwMjY5ZjJkMzAwMTUxOTRlNDlmMGI2YjFhMmI5YWZmZTZkMTRjNDY3NjJjNGEwZWRkNDI1ZjQxYTE3MT9yZXNwb25zZS1jb250

In [5]:
model_path = "./labse.Q8_0.gguf"
llm = Llama(model_path=model_path, embedding=True, n_gpu_layers=1)

llama_model_loader: loaded meta data with 24 key-value pairs and 197 tensors from ./labse.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = bert
llama_model_loader: - kv   1:                               general.name str              = LaBSE
llama_model_loader: - kv   2:                           bert.block_count u32              = 12
llama_model_loader: - kv   3:                        bert.context_length u32              = 512
llama_model_loader: - kv   4:                      bert.embedding_length u32              = 768
llama_model_loader: - kv   5:                   bert.feed_forward_length u32              = 3072
llama_model_loader: - kv   6:                  bert.attention.head_count u32              = 12
llama_model_loader: - kv   7:          bert.attention.layer_norm_epsilon f32              = 0.000000


In [6]:
import numpy as np

def get_embedding(text):
    output = llm.embed(text)
    return output

def cosine_similarity(v1, v2):
    v1_norm = np.linalg.norm(v1)
    v2_norm = np.linalg.norm(v2)
    if v1_norm == 0 or v2_norm == 0:
        return 0.0  # Avoid division by zero
    return np.dot(v1, v2) / (v1_norm * v2_norm)

def cos_dist(v1, v2):
    v1_norm = np.linalg.norm(v1)
    v2_norm = np.linalg.norm(v2)
    if v1_norm == 0 or v2_norm == 0:
        return None  # Avoid division by zero
    return 1 - (np.dot(v1, v2) / (v1_norm * v2_norm))



In [12]:
# Example usage
sentence0 = "今天天氣好熱。"
sentence1 = "The weather sure is hot today."
sentence2 = "今天天气好热。"

embedding0 = get_embedding(sentence0)
embedding1 = get_embedding(sentence1)
embedding2 = get_embedding(sentence2)

cos_dist01 = cos_dist(embedding0, embedding1)
cos_dist02 = cos_dist(embedding0, embedding2)

print(f"Cosine distance between sentence 0 and 1: {cos_dist01}")
print(f"osine distance between sentence 0 and 2: {cos_dist02}")

llama_perf_context_print:        load time =      51.28 ms
llama_perf_context_print: prompt eval time =      44.49 ms /     9 tokens (    4.94 ms per token,   202.29 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      48.96 ms /    10 tokens
llama_perf_context_print:        load time =      51.28 ms
llama_perf_context_print: prompt eval time =      43.19 ms /     9 tokens (    4.80 ms per token,   208.38 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      44.98 ms /    10 tokens
llama_perf_context_print:        load time =      51.28 ms
llama_perf_context_print: prompt eval time =      42.27 ms /     9 tokens (    4.70 ms per token,   212.91 tokens per second)
llama_perf_context_print:        eval time = 

Cosine distance between sentence 0 and 1: 0.15513960511426406
osine distance between sentence 0 and 2: 0.0027118204153381287
